In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
import sys
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'funcs'))
import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2

ind49_rets = erk.get_ind_returns(weighting="vw", n_inds=49)
fff = erk.get_fff_returns()

In [2]:
def get_capm(ind49_rets, fff, start, end, industry, full_fama=False):
    ind_rets = ind49_rets.get(industry)[str(start):str(end)]
    y_sm = ind_rets - fff['RF'][str(start):str(end)]
    mkt_excess = fff['Mkt-RF'][str(start):str(end)]
    x_sm = sm.add_constant(mkt_excess)
    if full_fama == True:
        x_sm['Value'] = fff['HML'][str(start):str(end)]
        x_sm['Size'] = fff['SMB'][str(start):str(end)]
    lm = sm.OLS(y_sm, x_sm)
    return lm.fit().params

## Question 1

In [3]:
get_capm(ind49_rets, fff, 1991, 2018, "Beer")

const     0.004294
Mkt-RF    0.529542
dtype: float64

## Question 2

In [4]:
get_capm(ind49_rets, fff, 1991, 2018, "Steel")

const    -0.00554
Mkt-RF    1.55461
dtype: float64

## Question 3

In [5]:
get_capm(ind49_rets, fff, 2013, 2018, "Beer")

const     0.004486
Mkt-RF    0.585960
dtype: float64

## Question 4

In [6]:
get_capm(ind49_rets, fff, 2013, 2018, "Steel")

const    -0.009797
Mkt-RF    1.416945
dtype: float64

## Question 5

In [7]:
betas = dict([(industry, get_capm(ind49_rets, fff, 1991, 1993, industry)["Mkt-RF"]) for industry in ind49_rets.columns])
betas = pd.DataFrame(betas, index=["Beta"]).transpose().sort_values("Beta")
betas.idxmax()

Beta    Hlth
dtype: object

## Question 6

In [8]:
betas.idxmin()

Beta    Gold
dtype: object

## Question 7

In [9]:
betas = dict([(industry, get_capm(ind49_rets, fff, 1991, 2018, industry, full_fama=True)["Size"]) for industry in ind49_rets.columns])
betas = pd.DataFrame(betas, index=["Size"]).transpose().sort_values("Size")
betas.idxmax()

Size    FabPr
dtype: object

## Question 8

In [10]:
betas.idxmin()

Size    Beer
dtype: object

## Question 9

In [11]:
betas = dict([(industry, get_capm(ind49_rets, fff, 1991, 2018, industry, full_fama=True)["Value"]) for industry in ind49_rets.columns])
betas = pd.DataFrame(betas, index=["Value"]).transpose().sort_values("Value")
betas.idxmax()

Value    Txtls
dtype: object

## Question 10

In [12]:
betas.idxmin()

Value    Softw
dtype: object